# Demonstration of Robust PCA using cvxpy

In [1]:
import cvxpy as cp
import numpy as np


In [2]:
# Construct a random 20-by-20 Gaussian matrix and construct a rank-1 matrix using its top-1 singular vectors
R = np.random.randn(20, 20)
U, S, Vt = np.linalg.svd(R, full_matrices=False)
L0 = np.outer(U[:, 0], Vt[0, :])

# Construct a 90% uniformly sparse matrix
E0 = np.random.rand(20, 20)
S0 = 1 * (E0 > 0.9)
X = L0 + S0

# Choose the regularization parameter
lambda_val = 0.25  # try different lambda #1/np.sqrt(20)


In [3]:
# Define and solve the cvxpy problem
L = cp.Variable((20, 20))
S = cp.Variable((20, 20))
W1 = cp.Variable((20, 20), symmetric=True)
W2 = cp.Variable((20, 20), symmetric=True)
Y = cp.Variable((40, 40), symmetric=True)

constraints = [
    L + S >= X - 1e-5,
    L + S <= X + 1e-5,
    Y == cp.bmat([[W1, L.T], [L, W2]]),
    Y >> 0  # PSD constraint
]

objective = cp.Minimize(0.5 * cp.trace(W1) + 0.5 * cp.trace(W2) + lambda_val * cp.pnorm(S, 1)) # note that np.norm(X_2,1) is matrix-1 norm, np.pnorm(X_2,1) is vectorized 1-norm.
prob = cp.Problem(objective, constraints)
prob.solve()

# The difference between sparse solution S and true S0
print('||S-S0||_infty:')
print(np.linalg.norm(S.value - S0, np.inf))
# The difference between the low rank solution L and true L0
print('||L-L0||:')
print(np.linalg.norm(L.value - L0))

||S-S0||_infty:
0.00017785020835689246
||L-L0||:
0.0001922178946376474


In [4]:
# Another simple cvxpy implementation directly using matrix nuclear norm
X_1 = cp.Variable((20, 20))
X_2 = cp.Variable((20, 20))

objective_nuc = cp.Minimize(cp.normNuc(X_1) + lambda_val * cp.pnorm(X_2, 1)) # note that np.norm(X_2,1) is matrix-1 norm, np.pnorm(X_2,1) is vectorized 1-norm.
constraints_nuc = [
    X_1 + X_2 == X
]
prob_nuc = cp.Problem(objective_nuc, constraints_nuc)
prob_nuc.solve(solver=cp.SCS, verbose=True)

# The difference between sparse solution X_2 and true S0
print('||X_2-S0||_infty:')
print(np.linalg.norm(X_2.value - S0, np.inf))
# The difference between the low rank solution X_1 and true L0
print('||X_1-L0||:')
print(np.linalg.norm(X_1.value - L0))

                                     CVXPY                                     
                                     v1.3.3                                    
(CVXPY) Mar 14 05:51:54 AM: Your problem has 800 variables, 1 constraints, and 0 parameters.
(CVXPY) Mar 14 05:51:54 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Mar 14 05:51:54 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Mar 14 05:51:54 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Mar 14 05:51:54 AM: Compiling problem (target solver=SCS).
(CVXPY) Mar 14 05:51:54 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing ->